Implimentation of Convolutipon Neural Network ON nmist Dataset
========================================
Importing tensoeflow library and mnist dataset 

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

Declare batch size and classes
========================================
In Case of mnist classes are 10 (0 to 9)
x = [None,784] because dataset have 28 X 28 size so 784 pixels in 1 image and Back and white so 2 dimensional

In [2]:
mnist = input_data.read_data_sets("/input", one_hot = True)

n_classes = 10
batch_size = 128

x = tf.placeholder('float',[None,784])
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /input\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /input\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /input\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /input\t10k-labels-idx1-ubyte.gz


method for 2D convolution and maxpool
========================================
here strides mean how many pixel we shift during convolution and ksize is size of kernal(2 X 2) and 'SAME' is convolution type 

In [3]:
def conv2d(x, W):
	return tf.nn.conv2d(x, W, strides = [1,1,1,1],padding = 'SAME')
	
def maxpool2d(x):
	return tf.nn.max_pool(x, ksize = [1,2,2,1],strides = [1,2,2,1], padding = 'SAME')

Defining Model
========================================
layer 1 will produce 32 output
layer 1 will produce 64 output
Fully connected layer 1024 output
output layer will classify it into 10 classes
(well you can play with no. of output)

In [4]:
def convolutional_neural_network(x):
	Weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
			   'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
	           'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
			   'out':tf.Variable(tf.random_normal([1024,n_classes]))}
			   
	biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
			   'b_conv2':tf.Variable(tf.random_normal([64])),
	           'b_fc':tf.Variable(tf.random_normal([1024])),
			   'out':tf.Variable(tf.random_normal([n_classes]))}
	
	x = tf.reshape(x ,shape = [-1,28,28,1])
	
	conv1 = tf.nn.relu(conv2d(x,Weights['W_conv1'])+biases['b_conv1'])
	conv1 = maxpool2d(conv1)
	
	conv2 = tf.nn.relu(conv2d(conv1,Weights['W_conv2'])+biases['b_conv2'])
	conv2 = maxpool2d(conv2)
	
	fc = tf.reshape(conv2,[-1,7*7*64])
	fc = tf.nn.relu(tf.matmul(fc,Weights['W_fc']))+biases['b_fc']

	fc = tf.nn.dropout(fc,keep_rate)
	
	output = tf.matmul(fc,Weights['out'])+biases['out']
			   
	return output

Training Part
========================================
To calculate cost used sofmax fuction which give probability
No. of epoch you can decide how much you want
AdamOptimizer I also don't know much about it

In [5]:
def train_neural_network(x):
	prediction = convolutional_neural_network(x)
	cost  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
	optimizer = tf.train.AdamOptimizer().minimize(cost) #default learning rate = 0.001
	
	hm_epochs = 25
	
	with tf.Session() as sess:
		sess.run(tf.global_variables_initializer())
		
		for epoch in range(hm_epochs):
			epoch_loss = 0
			for _ in range(int(mnist.train.num_examples/batch_size)):
				epoch_x,epoch_y = mnist.train.next_batch(batch_size)
				_, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x,y: epoch_y})
				epoch_loss += c
			print('Epoch', epoch+1, 'Completed out of ',hm_epochs,'loss :',epoch_loss )
			
		correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
		
		accuracy = tf.reduce_mean(tf.cast(correct,'float'))
		print('Accuracy',accuracy.eval({x : mnist.test.images, y : mnist.test.labels}))

In [ ]:
train_neural_network(x)

Output

![title](Pictures\cnn.png)